Import necessary libraries

In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

Import all data

In [123]:
seasons = range(2010,2023)
premier_match_data = {}
premier_team_data = {}
premier_player_data = {}

for i in seasons:
    match_data_file = f'..//data//england_premier_match//england-premier-league-matches-{i}-to-{i+1}-stats.csv'
    team_data_file = f'..//data//england_premier_team//england-premier-league-teams-{i}-to-{i+1}-stats.csv'
    player_data_file = f'..//data//england_premier_player//england-premier-league-players-{i}-to-{i+1}-stats.csv'

    premier_match_data[f'{i}_{i+1}'] = pd.read_csv(match_data_file)
    premier_team_data[f'{i}_{i+1}'] = pd.read_csv(team_data_file)
    premier_player_data[f'{i}_{i+1}'] = pd.read_csv(player_data_file)


Pre processing data
1. fill all missing value with -1 
2. Split goal scoring minutes into first half and second half
3. label encoding all object column
4. group previous three matches as the feature to train model.

In [115]:
def bin_goal_timings(goal_timings):
    # Parse the string into a list of integers
    if not isinstance(goal_timings, str):
        return (0,0)  # Return (0,0) if goal_timings is not a string

    if goal_timings=='-1':
        return (0,0)
    # Split the string into a list of times
    goal_times = goal_timings.split(',')

    first_half_goals = 0
    second_half_goals = 0

    for time in goal_times:
        # Check if it's stoppage time
        if "'" in time:
            time_parts = time.split("'")
            # Consider stoppage time as part of the second half
            if int(time_parts[0]) >= 45:
                second_half_goals += 1
        else:
            # Check if the goal was scored in the first or second half
            if int(time) <= 45:
                first_half_goals += 1
            else:
                second_half_goals += 1
    return first_half_goals, second_half_goals


In [124]:
# Create a list of all team names across all seasons
all_teams = []
all_referee = []
all_stadiums = []
for season in premier_match_data.keys():
    all_teams.extend(list(premier_match_data[season]['home_team_name'].unique()))
    all_referee.extend(list(premier_match_data[season]['referee'].unique()))
    all_stadiums.extend(list(premier_match_data[season]['stadium_name'].unique()))
    

In [125]:
le = LabelEncoder()
le_teams = LabelEncoder()
le_referee = LabelEncoder()
le_stadium = LabelEncoder()

le_teams.fit(all_teams)
le_referee.fit(all_referee)
le_stadium.fit(all_stadiums)


avoid_column = ['home_team_name','away_team_name','referee','stadium_name']
for season, df in premier_match_data.items():

    # Transform teams
    df['home_team_name'] = le_teams.transform(df['home_team_name'])
    df['away_team_name'] = le_teams.transform(df['away_team_name'])
    df['referee'] = le_referee.transform(df['referee'])
    df['stadium_name'] = le_stadium.transform(df['stadium_name'])


    premier_match_data[season] = df.fillna(-1,inplace=True)
    # split goal scoring time into first and second half
    df['first_half_goals_home'], df['second_half_goals_home'] = zip(*df['home_team_goal_timings'].apply(bin_goal_timings))
    df['first_half_goals_away'], df['second_half_goals_away'] = zip(*df['away_team_goal_timings'].apply(bin_goal_timings))
    
    # Convert the column to string type
    for col in df.columns:
        
        if df[col].dtype == 'object' and col not in avoid_column:
            # Convert the column to string type
            df[col] = df[col].astype(str)
            
            # Apply the label encoder
            le.fit(df[col])
            df[col] = le.transform(df[col])

    
    premier_match_data[season] = df
    

In [126]:
premier_match_data['2022_2023']

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,first_half_goals_home,second_half_goals_home,first_half_goals_away,second_half_goals_away
0,1659726000,37,0,25286.0,12,1,2,1,0.00,0.00,...,1.90,3.25,6.75,1.80,1.95,41,0,0,1,1
1,1659785400,38,0,-1.0,14,19,1,1,0.00,0.00,...,1.66,2.43,4.10,1.91,1.91,12,1,1,0,2
2,1659794400,39,0,-1.0,0,2,28,1,0.00,0.00,...,1.95,3.66,7.50,1.80,1.95,65,1,1,0,0
3,1659794400,39,0,-1.0,17,38,30,1,0.00,0.00,...,2.03,3.62,7.50,1.80,1.95,14,1,1,1,0
4,1659794400,39,0,52245.0,23,25,33,1,0.00,0.00,...,1.94,3.73,7.50,2.00,1.75,43,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1685287800,170,0,-1.0,13,0,34,38,1.00,0.94,...,1.67,2.63,4.50,1.91,1.91,21,0,1,0,0
376,1685287800,170,0,-1.0,17,33,2,38,1.22,1.11,...,1.44,2.30,3.75,1.44,2.63,14,0,1,1,3
377,1685287800,170,0,-1.0,18,36,33,38,0.89,0.67,...,1.61,2.60,4.50,1.62,2.20,27,1,1,0,1
378,1685287800,170,0,-1.0,21,14,30,38,2.50,1.28,...,1.40,2.10,3.50,1.62,2.20,37,1,1,1,0


Now we need to group previous 3 game stats as features of this game

In [127]:
def calculate_rolling_stats(group):
    # List of columns to skip
    columns_to_skip = ['timestamp', 'date_GMT', 'status', 'home_team_name', 'away_team_name', 'referee', 'Game Week']
    group = group.sort_values('Game Week')
    # Iterate over all columns in the group
    for col in group.columns:
        # Skip the column if it is in the list of columns to skip
        if col not in columns_to_skip:
            # The rolling window size is 3, which means the previous 3 games
            group['rolling_avg_' + col] = group[col].rolling(window=3).mean()

    # Shift the data down 1 so the current game's stats aren't included
    group = group.shift(1)

    # Drop the first two rows (which won't have any rolling data)
    group = group.iloc[3:]
    
    return group


In [128]:
# Initialize empty dictionaries to hold the stats for all seasons
all_seasons_home_stats = {}
all_seasons_away_stats = {}
# Loop through all seasons
for season in premier_match_data.keys():
    # Apply the calculate_rolling_stats function to the home and away data for this season
    all_seasons_home_stats[season] = premier_match_data[season].groupby('home_team_name').apply(calculate_rolling_stats)
    all_seasons_away_stats[season] = premier_match_data[season].groupby('away_team_name').apply(calculate_rolling_stats)

# Convert the dictionaries to dataframes
all_seasons_home_stats_df = pd.concat(all_seasons_home_stats, keys=all_seasons_home_stats.keys())
all_seasons_away_stats_df = pd.concat(all_seasons_away_stats, keys=all_seasons_away_stats.keys())

In [129]:
all_seasons_home_stats_df.to_csv("2022-23.csv", index=False)